In [1]:
import ee

In [2]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWiAUiCYR3M4WD01NG-8vc3GmAIU-2iRoQbzkqZuySbJnJ2ZZ_HHLMs

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [3]:
prudhoebay = ee.FeatureCollection("users/bibekaryal7/ABoVE_small/prudhoebay")
ivotuk_north = ee.FeatureCollection("users/bibekaryal7/ABoVE_small/ivotuk_north")
ivotuk_south = ee.FeatureCollection("users/bibekaryal7/ABoVE_small/ivotuk_south")
anwr = ee.FeatureCollection("users/bibekaryal7/ABoVE_small/anwr")

In [4]:
regionList = [prudhoebay, ivotuk_north, ivotuk_south, anwr]
nameList = ['prudhoebay', 'ivotuk_north', 'ivotuk_south', 'anwr']
crs_list = ['EPSG:26905', 'EPSG:26904', 'EPSG:26904', 'EPSG:26906']

In [5]:
modisNDVIlist = ee.ImageCollection('MODIS/006/MOD13A1').filter(ee.Filter.date('2000-02-18', '2021-07-29')).select("NDVI")

In [6]:
n_images = modisNDVIlist.size().getInfo()
modisNDVIlist = modisNDVIlist.toList(n_images)

In [7]:
for i in range(n_images): #Iterate through all images
  img = ee.Image(modisNDVIlist.get(i)) #get current image
  for j in range(len(regionList)): #Iterate through all regions
    clipped_image = img.clip(regionList[j]); #Clip image by region to reduce storage space
    clipped_image = clipped_image.reproject(crs_list[j]);
    img_name = clipped_image.get('system:index');
    img_name = img_name.getInfo();
    task = ee.batch.Export.image.toDrive(
        image=clipped_image,
        region=regionList[j].geometry(),
        description=img_name,
        folder=nameList[j],
        scale=30,
        maxPixels=3101797300
    )
    task.start()